# Recipe 1: Howso Engine Intro 

## Overview 

 
Howso Engine is a generalized Machine Learning (ML) and Artificial Intelligence platform that creates powerful decision-making models that are fully explainable, auditable, and editable. Howson Engine uses Instance-Based Machine Learning which stores instances, i.e., data points, in memory and makes predictions about new instances given their relationship to existing instances. This technology harnesses a fast spatial query system and information theory for performance and accuracy. 

 

Howso Engine is unique in that it utilizes a one-model approach (the data is the model) that enables the user to accomplish: 

 

- "Targetless Learning" - allows the user to predict and characterize any set of variables 

- Supervised Learning 

- Unsupervised Learning 

- Reinforcement Learning 

- Online Learning 

- Discriminative and Generative Modeling 

- Model Interpretation and Explanation 

- Data Imputation 

 

In this notebook we will explore the most basic workflow for training Howso Engine and making predictions. This will enable us to become familiar with basic Howso Engine terminologies, including: 

 

- Action Features 

- Context Features 

- Cases 

- Trainee 

- Train 

- Analyze 

- React 

## Recipe Goals:

This notebook will provide a demonstration of a basic Howso Engine workflow.

In [1]:
import pandas as pd
from pathlib import Path
from pmlb import fetch_data

from howso.engine import Trainee, load_trainee
from howso.utilities import infer_feature_attributes

# Step 1: Import Data

Our example dataset for this recipe is the well known `Adult` dataset. This dataset consists of 14 Context Features and 1 Action Feature. The Action Feature in this version of the `Adult` dataset has been renamed to `target` and it takes the form of a binary indicator for whether a person in the data makes more than $50,000/year (*target*=1) or less (*target*=0).

**`Definitions`:**

**`Action Features`:** A set of Features that is used as a schema to describe values for the output of a Trainee when it Reacts. In traditional ML these are often referred to as targets, target features, dependent features, or labels.

**`Context Features`:** A set of Features that is used as a schema to describe values for input, observation, or other computation. In traditional ML these are often referred to as inputs, predictors, or independent features.



In [2]:
df = fetch_data('adult', local_cache_dir="data/adult")

# Subsample the data to ensure the example runs quickly
df = df.sample(2000)

df

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,target
2882,53.0,4,196328.0,9,13.0,2,4,0,2,1,0.0,0.0,40.0,39,1
17999,27.0,4,137063.0,9,13.0,2,12,0,4,1,0.0,0.0,50.0,39,1
27549,20.0,0,232022.0,15,10.0,4,0,3,4,0,0.0,0.0,40.0,39,1
15163,25.0,4,207875.0,15,10.0,4,5,3,4,1,0.0,0.0,20.0,39,1
12422,17.0,4,413557.0,1,7.0,4,1,3,4,0,0.0,0.0,32.0,39,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40835,38.0,4,365307.0,11,9.0,5,1,1,4,0,0.0,0.0,32.0,39,1
37207,81.0,0,143732.0,3,2.0,6,0,1,4,1,0.0,0.0,30.0,39,1
41058,35.0,4,195946.0,7,12.0,2,8,0,1,1,0.0,0.0,40.0,37,1
10778,24.0,4,142022.0,15,10.0,4,6,3,4,1,0.0,0.0,50.0,39,1


# Step 2: Feature Mapping

Typically, an exploratory analysis is done on the data to get a general feel of the descriptive statistics and data attributes. 

Methods like `describe` from a Pandas dataframe often automatically present these types of information of interest to a user, as shown below. While informative, these descriptive statistics are often used as a sanity check pre- and post-modeling and a model typically doesn't actually use any of these feature attributes.

In [3]:
df.describe()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,target
count,2000.000000,2000.000000,2000.00000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000
mean,37.870500,3.852000,187990.78800,10.204000,10.081000,2.680500,6.659000,1.470000,3.644500,0.669000,1241.385000,90.538000,40.406500,37.172000,0.762500
std,13.625952,1.472134,107646.26818,3.959953,2.518255,1.489809,4.231746,1.606359,0.884031,0.470691,8390.585727,407.977809,12.326038,6.922527,0.425658
min,17.000000,0.000000,19752.00000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000
25%,27.000000,4.000000,114867.25000,9.000000,9.000000,2.000000,3.000000,0.000000,4.000000,0.000000,0.000000,0.000000,40.000000,39.000000,1.000000
50%,36.000000,4.000000,177538.50000,11.000000,10.000000,2.000000,7.000000,1.000000,4.000000,1.000000,0.000000,0.000000,40.000000,39.000000,1.000000
75%,47.000000,4.000000,234828.00000,12.000000,12.000000,4.000000,10.000000,3.000000,4.000000,1.000000,0.000000,0.000000,45.000000,39.000000,1.000000
max,90.000000,8.000000,972354.00000,15.000000,16.000000,6.000000,14.000000,5.000000,4.000000,1.000000,99999.000000,3770.000000,99.000000,40.000000,1.000000


In the Howso Engine workflow, feature attributes are an essential part of model building and usage. By incorporating certain feature attributes into training process itself, Howso Engine gains another layer of information that will help in fine-tuning the results. 

In order to assist the user with defining the feature attributes, Howso has an `infer_feature_attributes` tool that automatically processes the dataset for the user.

In Howso Engine, these feature attributes are model infrastructure feature parameters that are based on the existing data, rather than exact descriptive statistics. This is why, for example, the min and max bounds on continuous features are not the exact min and max values of the dataset, but rather an expanded version of those min and max values to allow for some variation.

In [4]:
# Infer features attributes
features = infer_feature_attributes(df)
features

{'age': {'type': 'continuous',
  'decimal_places': 0,
  'original_type': {'data_type': 'numeric', 'size': 8},
  'bounds': {'min': 7.0, 'max': 148.0}},
 'workclass': {'type': 'nominal',
  'data_type': 'number',
  'decimal_places': 0,
  'original_type': {'data_type': 'integer', 'size': 8},
  'bounds': {'allow_null': False}},
 'fnlwgt': {'type': 'continuous',
  'decimal_places': 0,
  'original_type': {'data_type': 'numeric', 'size': 8},
  'bounds': {'min': 8103.0, 'max': 1202604.0}},
 'education': {'type': 'nominal',
  'data_type': 'number',
  'decimal_places': 0,
  'original_type': {'data_type': 'integer', 'size': 8},
  'bounds': {'allow_null': False}},
 'education-num': {'type': 'continuous',
  'decimal_places': 0,
  'original_type': {'data_type': 'numeric', 'size': 8},
  'bounds': {'min': 1.0, 'max': 20.0}},
 'marital-status': {'type': 'nominal',
  'data_type': 'number',
  'decimal_places': 0,
  'original_type': {'data_type': 'integer', 'size': 8},
  'bounds': {'allow_null': False}},
 

In [5]:
# Specify Context and Action Features
action_features = ['target']
context_features = features.get_names(without=action_features)


> **Note:** Train Test Split

To gauge model performance, train-test splits are often used in traditional machine learning workflows. Howso Engine does not require the use of train-test split for validation. Please see recipe `6-validation.ipynb` for further explanation. Therefore, we will not use train-test splits our recipes unless the test set serves a specific purpose.

# Step 3: Create Trainee

To begin the Howso Engine workflow, a Trainee is created to act as a base for all of our ML needs. In all subsequent notebooks and jupyter cells, we will refer to Howso Engine's model as Trainee.

**`Definitions`:**

**`Trainee`:** A collection of Cases that comprise knowledge. May include metadata and parameters. In traditional ML this is referred to as a model.

**`Cases`:** A set of feature values representing a situation observed.  In traditional ML, a Case is sometimes referred to as an "observation", "record", or "data point". In database terms, a Case would be a row of values. For supervised learning a Case is a set of Context Features and Action Features and for unsupervised learning a Case is just a set of features. 

In [6]:
# Create the Trainee
t = Trainee(
    features=features,  
    overwrite_existing=True
)

# Step 4: Preprocessing and Training

One benefit of Howso Engine is that most standard forms of data pre-processing such as one hot encoding and standardizing are `NOT` needed, which is in contrast to many traditional ML models. This does not include more sophisticated forms of pre-processing such as feature selection or feature engineering, which may still be useful. Fitting is also done in two steps in Howso Engine.

**`Definitions`:**

**`Train`:** Exposing a Trainee to a Case which may cause the ML algorithm to update the Trainee. This is a single training step; training may happen at each decision, at a certain sampling rate of observations per second, or at certain events.

**`Analyze`:** Tune internal parameters to improve performance and accuracy of predictions and metrics. Analysis may be targeted or targetless.  Targetless analysis provides the best balanced set of hyperparameters if an Action Feature is not specified, along with a performance boost while targeted analysis provides a boost to accuracy. 


In [7]:
# Train
t.train(df)

# Analyze the Trainee
t.analyze(context_features=context_features, action_features=action_features)

# Step 5: Results

Once Howso Engine is trained and analyzed, it provides the user with a variety of ML capabilities. At this stage in the Howso Engine workflow, a typical use case would be to evaluate the accuracy of the Trainee, which is performed by the `react` method. This is equivalent to `predict` in many traditional Machine Learning workflows, although the `react` method is not solely used for supervised predictions as detailed in subsequent recipes. 

Since we are not using a train-test split, we will use the `react_into_trainee` method, which performs a `react` on each of the cases that is trained into the model. Alternatively, a standard `react` call may be used on unseen data for prediction.
The accuracy is calculated internally as shown in the code below and this is the recommended accuracy metric. Further explanations are available in recipe `6-validation`.

**`Definitions`:**

**`React`:** Exposing a Trainee to a new Case's Context Features and an Action Feature for that case is returned. In traditional ML this is often referred to predicting or labeling.

In [8]:
# Recommended metrics
t.react_into_trainee(action_feature=action_features[0], residuals=True)
stats = t.get_prediction_stats().target.round(2)
print(f'Howso Prediction Results - Accuracy: {stats["accuracy"]}, Precision: {stats["recall"]}, and Recall: {stats["precision"]}')

Howso Prediction Results - Accuracy: 0.83, Precision: 0.78, and Recall: 0.73


In [9]:
stats_matrix = t.get_prediction_stats(stats=["confusion_matrix"])
print("Howso Prediction Results - Confusion Matrix")
matrix = pd.DataFrame(stats_matrix.loc["confusion_matrix", "target"])
matrix.index.name = "Predicted"
matrix.columns.name = "Actual"
display(matrix)

Howso Prediction Results - Confusion Matrix


Actual,1,0
Predicted,,
1,708,113
0,54,125


Another way to use the `get_prediction_stats` method is to utilize the `condition` parameter. Specifying a condition allows the user to get prediction stats on the subset of cases that matches the specified condition.

For example, we can use the `condition` parameter to find the accuracy, precision, and recall on the subsets of the dataset with each value for Sex.

In [10]:
sex_1_stats = t.get_prediction_stats(condition={'sex': 1})['target'].round(2)
print(f'Howso Prediction Results On Cases with Sex=1 - Accuracy: {sex_1_stats["accuracy"]}, Precision: {sex_1_stats["recall"]}, and Recall: {sex_1_stats["precision"]}')

sex_0_stats = t.get_prediction_stats(condition={'sex': 0})['target'].round(2)
print(f'Howso Prediction Results On Cases with Sex=0 - Accuracy: {sex_0_stats["accuracy"]}, Precision: {sex_0_stats["recall"]}, and Recall: {sex_0_stats["precision"]}')

Howso Prediction Results On Cases with Sex=1 - Accuracy: 0.78, Precision: 0.76, and Recall: 0.72


Howso Prediction Results On Cases with Sex=0 - Accuracy: 0.92, Precision: 0.8, and Recall: 0.7


# Step 6: Saving, Loading, and Deleting from your local disk

These methods work for `HowsoDirectClient` and its subclasses.

## Saving and Loading

When saving to disk, a filepath or filename may provided. The filepath provided may be a relative or absolute path. The filepath can include the desired filename with a `.caml` extension. If no filename is at the end of the filepath, the Trainee ID will be used. If just the filename is provided, then the current working directory will be used as the filepath. 

> **Note:** It is recommended, however not necessary that the filename match the trainee ID


## Deleting

When deleting a Trainee, the `delete` method deletes the trainee from the last saved or loaded disk location, as well as memory. If the Trainee has not been saved, `delete` can also be used to just remove a Trainee from memory.

When Trainees are saved, a `.caml` file and a `.txt` version file is saved. The `delete` method assumes that the prefix to the version filename is the same as the `.caml` trainee filename.

In [11]:
from howso.direct import HowsoDirectClient

# Clients that extend `HowsoDirectClient` will have local file operations
if isinstance(t.client, HowsoDirectClient):
    # Example filepath (same as default)
    cwd = Path.cwd()
    file_path=f"{cwd}/engine_intro_trainee.caml"

    # Saving
    t.save(file_path=file_path) 

    # Loading
    t = load_trainee(file_path=file_path)

    # Deleting
    t.delete()

# Conclusion and Next Steps

Howso Engine provide users with powerful tools to perform a variety of data analysis and prediction tasks. However, making predictions may not suffice if these tools are used for decision-making. As an example with this specific `Adult` dataset, if the user were a bank predicting whether an individual will make over or under $50k in the future for loan decisions, the outcome of prediction efforts might greatly affect the livelihood of that individual as well as the bank itself, if the prediction was inaccurate or biased. Ideally, a predictive tool should provide both accurate results and interpretability so that it can be held accountable for its predictions. Unfortunately, traditional ML techniques often leave a lot to be desired in terms of interpretability. Because of its instance-based ML nature, Howso Engine is inherently interpretable.

Thus, the next step to fully utilizing Howso Engine is to take advantage of its interpretability capabilities, which will be introduced in recipe `2-interpretability.ipynb`.

